In [28]:
%load_ext autoreload
%autoreload 2

from meds_pipeline import Tokenizer
import pickle
import polars as pl
import numpy as np
from stream_reconstructor import StreamReconstructor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Loading tokenizer
tok_path = '/nfs/roberts/project/pi_ajl89/kam385/mimic-iv/v1/tokenized/concept_discrete_factored/tok.pkl'
with open(tok_path, "rb") as f:
    tok = pickle.load(f)

tok.itot
# tok.ttoi

{0: '0DJ03ZZ',
 1: 'OTHER OR PROCEDURES OF THE BLOOD AND BLOOD FORMING ORGANS WITH MCC',
 2: '447',
 3: '0PST3ZZ',
 4: '0C9G00Z',
 5: '0MTL0ZZ',
 6: 'S61205A',
 7: 'M4200',
 8: 'HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SYSTEM W/O MCC',
 9: 'V4919XA',
 10: 'M8669',
 11: '0X6R0Z1',
 12: '0TY10Z1',
 13: 'S06309S',
 14: 'A07BA51',
 15: 'Q820',
 16: '0B5D3ZZ',
 17: '07TB4ZZ',
 18: '047N341',
 19: '00160J6',
 20: 'FRACTURE OF PELVIS OR DISLOCATION OF HIP',
 21: 'C250',
 22: 'C413',
 23: 'DB023ZZ',
 24: 'ENDOCRINE DISORDERS WITHOUT CC/MCC',
 25: '0HN7XZZ',
 26: 'S3739XA',
 27: '0WPF0KZ',
 28: '220051mmHg',
 29: 'A4153',
 30: 'Not Started per Sliding Scale',
 31: '0T9030Z',
 32: 'H7012',
 33: 'S68620A',
 34: 'P0418',
 35: '03H433Z',
 36: 'S62327A',
 37: 'J150',
 38: 'R402221',
 39: 'A5215',
 40: 'J684',
 41: '0JH60MZ',
 42: '0J2TXYZ',
 43: 'S36114A',
 44: 'R2681',
 45: 'N9489',
 46: '0LB60ZZ',
 47: 'T461X6A',
 48: 'LYMPHOMA & NONACUTE LEUKEMIA W/O CC/MCC',
 49: 'I7773',
 50: 'X34XXXA',
 51:

In [4]:
# show bins, if wanted
tok.bin_params['<TIME>']

{'edges': [1.0,
  60.0,
  360.0,
  900.0,
  1979.0,
  3540.0,
  6120.0,
  17760.0,
  2903205240.0]}

In [6]:
#Loading binary of tokenized event sequence
bin_path = '/nfs/roberts/project/pi_ajl89/kam385/mimic-iv/v1/tokenized/concept_discrete_factored/test.bin'
data = np.memmap(bin_path, dtype=np.uint16, mode='r')
data

memmap([38631, 11437, 21810, ..., 38636, 14058, 19392],
       shape=(93849910,), dtype=uint16)

In [37]:
for i in data[:20]:
    print(f'Token: {i} -> Code: {tok.itot[i]} ')

Token: 38631 -> Code: <sos> 
Token: 11437 -> Code: <GENDER> 
Token: 21810 -> Code: M 
Token: 15992 -> Code: <MEDS_BIRTH> 
Token: 38642 -> Code: <TIME> 
Token: 38640 -> Code: Q8 
Token: 2396 -> Code: <TRANSFER_TO> 
Token: 5533 -> Code: ED 
Token: 8537 -> Code: Emergency Department 
Token: 38631 -> Code: <sos> 
Token: 11437 -> Code: <GENDER> 
Token: 36186 -> Code: F 
Token: 15992 -> Code: <MEDS_BIRTH> 
Token: 38642 -> Code: <TIME> 
Token: 38640 -> Code: Q8 
Token: 20034 -> Code: <LAB> 
Token: 14433 -> Code: Blood Pressure Systolic 
Token: 38639 -> Code: Q7 
Token: 2859 -> Code: Blood Pressure Diastolic 
Token: 38639 -> Code: Q7 


In [38]:
sr = StreamReconstructor(tokenizer_vocab=tok.ttoi, bins=tok.bin_params) #if you were using a continuous tokenization method, you would have to pass scaling_params=tok.scaling_params instead of bins=tok.bin_params
t, x, v = sr.reconstruct(data)

In [41]:
print(t[200:210])
print(x[200:210])

[170937000000, 170937000000, 170937000000, 170937000000, 170937000000, 170937000000, 170937000000, 170937000000, 170937000000, 170937000000]
[18996, 38633, 25672, 38639, 28081, 38635, 10655, 38636, 14479, 38637]


In [42]:
for i in x[:20]:
    print(f'Token: {i} -> Code: {tok.itot[i]} ')

Token: 11437 -> Code: <GENDER> 
Token: 21810 -> Code: M 
Token: 15992 -> Code: <MEDS_BIRTH> 
Token: 20034 -> Code: <LAB> 
Token: 14433 -> Code: Blood Pressure Systolic 
Token: 38638 -> Code: Q6 
Token: 2859 -> Code: Blood Pressure Diastolic 
Token: 38639 -> Code: Q7 
Token: 12582 -> Code: <PROCEDURE> 
Token: 23306 -> Code: ICD 
Token: 17867 -> Code: 10 
Token: 36551 -> Code: 0DV44CZ 
Token: 2396 -> Code: <TRANSFER_TO> 
Token: 37137 -> Code: admit 
Token: 16444 -> Code: Discharge Lounge 
Token: 33548 -> Code: <HOSPITAL_ADMISSION> 
Token: 21061 -> Code: SURGICAL SAME DAY ADMISSION 
Token: 23427 -> Code: PHYSICIAN REFERRAL 
Token: 2396 -> Code: <TRANSFER_TO> 
Token: 16438 -> Code: transfer 
